<a href="https://colab.research.google.com/github/chr1shr/am205_g_activities/blob/master/dae_solver/dae_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The DAE solver class
The code below provides a skeleton for a DAE solver class. When completed, the user should be able to call the method ```integrate```, which integrates the supplied function ```fun``` with jacobian ```jac``` over a specified time range and step size. The solver calls a pseudo-private, internal method ```newton```, which solves a root-finding problem at each integration step. Following the notes from the session, implement the routine for Newton's method to complete the solver.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

class dae_solver(object):
    def __init__(self):
        ''' Constructor for the general-purpose DAE solver using the BDF method.'''
        # Backward differentiation formula coefficients.
        self.a = [[-1.],                        # 1st order
                  [1./3, -4./3],                # 2nd order
                  [-2./11, 9./11, -18./11]]     # 3rd order           
        self.b = [1., 2./3, 6./11]

    def integrate(self, m, n, fun, jac, yi, ti, tf, dt):
        ''' Routine to run DAE integration.
            m,n - the number of differential and algebraic degrees of freedom, respectively.
            fun, jac - the system of equations to integrate, and its jacobian.
            yi - the initial condition.
            ti, tf - initial and final integration times.
            dt - the step size to use.
        '''
        
        # Save the number of differential and algebraic variables as class attributes.
        self.m = m
        self.n = n

        # Compute number of integration steps to pre-allocate memory for the solution output:
        nsteps = int(np.round((tf-ti)/dt))+1
        t = np.zeros(nsteps)
        sol = np.zeros((nsteps, m+n))
        
        # Set initial condition.
        sol[0] = yi

        # First step: 1st order BDF
        sol[1] = self.__newton(1, fun, jac, ti+dt, dt, sol[0], sol[0:1])
        t[1] = ti + dt

        # Second step: 2nd order BDF
        sol[2] = self.__newton(2, fun, jac, ti+2*dt, dt, sol[1], sol[0:2])
        t[2] = ti + 2*dt

        # Resume with 3rd order BDF
        k = 2
        while ti + (k+1)*dt <= tf:
            sol[k+1] = self.__newton(3, fun, jac, ti+(k+1)*dt, dt, sol[k], sol[k-2:k+1])
            t[k+1] = ti + (k+1)*dt
            k += 1

        return t, sol

    # Double underscore declares a "pseudo-private" method,
    # accessible only within the DAE solver class.
    def __newton(self, p, fun, jac, t, dt, yi, ypre, tol=1.0e-14, maxiter=100):
        '''Newton's method for nonlinear root-finding.
           p - the order of method to use.
           fun - the function whose root to find, with signature fun(t,y).
           jac - the Jacobian of fun, with signature jac(t,y).
           t - the integration time to solve for.
           dt - the integration step size.
           yi - initial guess for solution vector.
           ypre - the previous solutions needed for the multistep method. 
           tol - stopping tolerance for residual.
           maxiter - maximum number of iterations allowed for convergence.
        '''
        # Define some shorthands for class variables:
        m = self.m # number of differential equations
        n = self.n # number of algebraic equations
        # BDF coefficients for order p
        alpha = np.array(self.a[p-1]) 
        beta = self.b[p-1]

        # your code here #
        

# Test problem: The simple pendulum

Test your DAE solver class by integrating the differential-algebraic system of equations for the simple pendulum, defined below:

In [ ]:
# gravitational constant
g = 9.80665

# simple pendulum DAE
def dae_fun(t, q):
    x, y, u, v, l = q[0], q[1], q[2], q[3], q[4]
    f = np.array([u,
                  v,
                  -l*x,
                  -(l*y + g),
                  x*x + y*y - 1])
    return f

def dae_jac(t, q):
    x, y, u, v, l = q[0], q[1], q[2], q[3], q[4]
    j = np.array([[  0,   0,   1,   0,   0 ],
                  [  0,   0,   0,   1,   0 ],
                  [ -l,   0,   0,   0,   -x],
                  [  0,  -l,   0,   0,   -y],
                  [ 2*x, 2*y,  0,   0,    0]])
    return j

Integrate the simple pendulum on $t=[0,10]$ with step size $h=10^{-3}$, with the following initial conditions:

$$
\begin{align}
\theta &= \pi/4 \\
x &= \sin\theta \\
y &= -\cos\theta \\
u &= v = 0 \\
\lambda &= g\cos\theta
\end{align}
$$
Here, $\lambda$ has been determined consistently using the last hidden constraint shown in class.

In [ ]:
# Set consistent initial conditions.
theta = np.pi/4.
x = np.sin(theta)
y = -np.cos(theta)
u = 0.; v = 0.
lam = g*np.cos(theta)
q0 = np.array([x, y, u, v, lam])

# Create an instance of the DAE solver class.
solver = dae_solver()

# Call the integrate method.
ndiff = 4; nalg = 1
t, sol = solver.integrate(ndiff, nalg, dae_fun, dae_jac, q0, 0., 10., 1e-3)

# Plot the variables as a function of t (columns of the array sol).

# your code here

You can optionally view a movie of the pendulum by using the provided script ```animate.py``` from the [AM 205 group activities Github page](https://github.com/chr1shr/am205_g_activities/tree/master/dae_solver). The following cell block imports the method ```animate_simple_pendulum``` which takes the ```sol``` array as an argument, and combines frames into a movie. To run the code below, make sure you have the script ```animate.py``` saved locally on your computer; you will be prompted to select it as a file to upload. **Running this may take a few minutes.**

In [ ]:
# upload animate.py
from google.colab import files
files.upload()

from animate import animate_simple_pendulum
from IPython.display import HTML

ani = animate_simple_pendulum(sol[::10])
HTML(ani.to_html5_video())

**Optional:** In the function and Jacobian for the simple pendulum, replace the algebraic constraint on length (index 3 formulation) with either the index 2 or index 1 constraint formulations. Integrate the DAE and plot the three constrained quantities. Do you see evidence of higher fluctuation or drift in the constraints that are not explicitly enforced?